In [1]:
import os
import import_ipynb
import polars as pl
import general_fns as gf
import coingecko_api as ca
import owlracle_api as oa

importing Jupyter notebook from general_fns.ipynb
importing Jupyter notebook from coingecko_api.ipynb
importing Jupyter notebook from owlracle_api.ipynb


In [2]:
# List of stablecoin symbols and IDs
syms = ['usdt', 'usdc', 'busd', 'dai', 'tusd', 'usdp', 'gusd']
sids = ['tether', 'usd-coin', 'binance-usd', 'dai', 'true-usd', 'paxos-standard', 'gemini-dollar']

# Variables for API call
currency = 'usd'
start_date = '2019/01/01'
end_date = '2022/12/31'

In [3]:
file_path = os.path.join("data/stablecoin/", "api-data.xlsx")

# Check if file already exists
if not os.path.exists(file_path):
    # Call the API and create dict of dataframes
    api_data = ca.create_api_dict(syms, sids, currency, start_date, end_date)
    # Create Excel file with API data
    gf.create_excel(file_path, api_data)

In [4]:
file_path = os.path.join("data/stablecoin/", "fees-data.xlsx")

# Check if file already exists
if not os.path.exists(file_path):
    # Call the API and create dict of dataframes
    fees_data = oa.create_df(start_date, end_date)
    # Create Excel file with API data
    gf.create_excel(file_path, fees_data)